# Localisation de la source d'émission d'une onde gravitationnelle #

Le programme proposé dans ce Notebook a pour objectif la localisation de la source d'une onde gravitationnelle à partir de données issues des détecteurs placés sur Terre.
Vous pouvez l'utiliser comme alternative au téléchargement du programme et son exécution dans un environnement Python sur votre ordinateur personnel.

## Programme à compléter avec les données des détecteurs ##
Code : Cédric Vanden Driessche (avec l'aide de Nicolas Arnaud LAL,Orsay / VIRGO)<br>
Pour exécuter ce programme, il vous suffit de cliquer dans la cellule ci-dessous et d'utiliser la commande Exécuter de la barre de menu, ou la combinaison de touches MAJ + ENTRÉE.

In [ ]:
#Code Cédric Vanden Driessche (avec l'aide de Nicolas Arnaud LAL,Orsay / VIRGO)
# Travail basé sur la thèse "Contrôle global de la partie centrale du détecteur
# d’ondes gravitationnelles Virgo. Recherche de signaux
# impulsionnels: application aux coincidences entre interféromètres"
# Auteur : Nicolas Arnaud

import numpy as np

print('#############################################################################')
print('####    Programme localisation d une source d onde gravitationnelle    ######')
print('#############################################################################')
print(' ')
      
      
#Constantes physiques
R=6371000
c=299792458 
Tterre=23*3600+56*60

#############################################
## Calcul Temps sidéral
def gps_to_Greenwich_mean_sideral_time(gps):
    gps0 = 630763213 # GPS of J2000 epoch: 2000 Jan 12, 12:00 UTC
    D = (gps - gps0) / 86400 # Number of days since J2000 epoch

    # (int+1/2) Days between J2000 and last 0h at Greenwich 
    d_u = np.floor(D) + 0.5
    if D - np.floor(D) < 0.5:
        d_u = d_u - 1

    df_u = D - d_u # Fraction of day since last 0h at Greenwich

    # GMST (s) at Greenwich at last 0h
    T_u = d_u / 36525;
    gmst0h = 24110.54841 + 8640184.812866 * T_u + 0.093104 * np.square(T_u) - 6.2e-6 * np.power(T_u, 3)
    
    # GMST (s)
    gmst = gmst0h + 1.00273790935 * 86400 * df_u
    # Remove integer days if any
    if gmst >= 86400:
        gmst = gmst - np.floor(gmst / 86400) * 86400
    
    # Conversion to radians
    tmp = gmst / 86400 * 2 * np.pi

    return(tmp)

#############################################
    
#Conversion degré en heure minutes secondes
def deg2hms(dd):
    """deg2dms(dd): conversion degrés décimaux dd -> liste [heure, minutes, secondes]"""
    L=[]
    L.append(int(dd*24/360))
    L.append(int((dd*24/360-L[0])*60))
    L.append((((dd*24/360-L[0])*60)-L[1])*60)
    return L    



#Mesures expérimentales retards entre détecteurs 6ms et 14 ms à 12h30m43s--> 10h13m12s en Ts https://www.webastro.net/ephemerides/conversion_temps/#resultat
#T=10*3600+13*60+12
    
#Lattitudes et logitudes des détecteurs
print('#############################################################################')
print('###########################       Position des détecteurs  ##################')
print('#############################################################################')
longitude_V1 = np.radians(float(input('Entrer la longitude de Virgo en degré : ')))
latitude_V1  = np.radians(float(input('Entrer la latitude de Virgo en degré : ')))

longitude_H1 = np.radians(float(input('Entrer la longitude de LIGO Hanford en degré : ')))
latitude_H1  = np.radians(float(input('Entrer la latitude de LIGO Hanford  en degré : ')))

longitude_L1 = np.radians(float(input('Entrer la longitude de LIGO Livingstone en degré : ')))
latitude_L1  = np.radians(float(input('Entrer la latitude de LIGO Livingstone en degré : ')))

#valeurs entrées automatiquement pour tester le programme
#longitude_V1 = np.radians(-10.5)
#latitude_V1  = np.radians(43.63)

#longitude_H1 = np.radians(119.41)
#latitude_H1  = np.radians(46.45)

#longitude_L1 = np.radians(90.77)
#latitude_L1  = np.radians(30.56)


# GW170814
print('')
print('#############################################################################')
print('###############      Mesures expérimentales pour GW170814  ##################')
print('#############################################################################')

GPS_event = 1186741861.527  
Delta12=float(input('Entrer le retard entre Virgo et Hanford en s : '))
Delta13=float(input('Entrer le retard entre Virgo et Livingstone en s : '))

Tsideral= gps_to_Greenwich_mean_sideral_time(GPS_event) 

#Vecteur position des 3 detecteurs, 1=Virgo, 2=LIGO H, 3=LIGO L
OD1=np.array([R*np.cos(latitude_V1)*np.cos(longitude_V1),R*np.cos(latitude_V1)*np.sin(longitude_V1),R*np.sin(latitude_V1)])
OD2=np.array([R*np.cos(latitude_H1)*np.cos(longitude_H1),R*np.cos(latitude_H1)*np.sin(longitude_H1),R*np.sin(latitude_H1)])
OD3=np.array([R*np.cos(latitude_L1)*np.cos(longitude_L1),R*np.cos(latitude_L1)*np.sin(longitude_L1),R*np.sin(latitude_L1)])

#Calcul vecteurs détecteur à détecteur
D12=OD2-OD1
D13=OD3-OD1
D23=OD3-OD2

#Calcul temps max entre détecteurs pour vérification avec données théoriques
print('')

print('#############################################################################')
print('################### Procédure de vérification ###############################')
print('#############################################################################')
      

print('Les valeurs théoriques du temps mesurables maximales entre 2 détecteurs \
       1=VIRGO 2=Hanford 3=Livingston) sont :')
print('')
print('D12max=27.2 ms; D13max=26.39 ms; D23max=10 ms')
print('')
print('vérification temps maximal entre 2 détecteurs avec les lattitudes et  \
      longitudes fournies')
print('')
NormeD12=np.linalg.norm(D12)
D12max=NormeD12/c
print('D12max=', D12max,'s')

NormeD13=np.linalg.norm(D13)
D13max=NormeD13/c
print('D13max=', D13max,'s')

NormeD23=np.linalg.norm(D23)
D23max=NormeD23/c
print('D23max=', D23max,'s')

print('')


#Construction de la base locale (u,v,w) avec Virgo pour origine
#u
u=D12/NormeD12

#v
Lambda=-(np.vdot(D12,D13))/NormeD12
vprime=Lambda*u+D13
Normevprime=np.linalg.norm(vprime)
v=vprime/Normevprime

#w
w=np.cross(u,v)

#vérification orthogonalité de la base
uv=np.vdot(u,v)
uw=np.dot(u,w)
vw=np.dot(v,w)

#print('u=',u,'v=',v,'w=',w)

#D12, D13 et n dans (u,v,w)
#D12 dans (u,v,w)
newD12=np.array([np.vdot(D12,u),np.vdot(D12,v),np.vdot(D12,w)])
#D13 dans (u,v,w)
newD13=np.array([np.vdot(D13,u),np.vdot(D13,v),np.vdot(D13,w)])
#n dans (u,v,w)
X=(Delta12*c)/newD12[0]
Y=((Delta13*c)-newD13[0]*X)/newD13[1]
Z=np.sqrt(1-X**2-Y**2)

n_plus=np.array([X,Y,Z])
n_moins=np.array([X,Y,-Z])

#print('a=',newD12[0],'b=',newD13[0],'c=',newD13[1])

#print('X=',X,'Y=',Y,'Z=+/-',Z)

#On applique à n la matrice de Rotation réseau-Greenwich
MRG=np.array([[-0.819998,0.494587,0.288075],[-0.571722,-0.731649,-0.371246],[0.0271568,-0.46912,0.882717]])

LV=np.radians(43.63)
#MRG=np.array([[np.sin(LV)*np.cos(Tsideral),np.sin(LV)*np.sin(Tsideral),-np.cos(LV)],[-np.sin(Tsideral),np.cos(Tsideral),0],[np.cos(LV)*np.cos(Tsideral),np.cos(LV)*np.sin(Tsideral),np.sin(LV)]])
Pos_plus=np.dot(MRG,n_plus)
Pos_moins=np.dot(MRG,n_moins)


#Pos_plus=n_plus
#Pos_moins=n_moins

#Calcul azimut et élévation
delta_plusR=np.arcsin(Pos_plus[2])
delta_plus=np.arcsin(Pos_plus[2])*180/np.pi     #delta en degre
ht_plusR=np.arccos(Pos_plus[0]/np.cos(delta_plusR))
ht_plus=np.arccos(Pos_plus[0]/np.cos(delta_plusR))*180/np.pi  #h(t) en degre

delta_moinsR=np.arcsin(Pos_moins[2])
delta_moins=np.arcsin(Pos_moins[2])*180/np.pi     #delta en degre
ht_moinsR=np.arccos(Pos_moins[0]/np.cos(delta_moinsR))
ht_moins=np.arccos(Pos_moins[0]/np.cos(delta_moinsR))*180/np.pi  #h(t) en degre

Tsideral_degre=(Tsideral/np.pi)*180
#TtD=(T/Tterre)*360
alpha_plus=Tsideral_degre-ht_plus
alpha_moins=Tsideral_degre-ht_moins

alpha_plusHMS=deg2hms(alpha_plus)
alpha_moinsHMS=deg2hms(alpha_moins)

print('')
print('#############################################################################')
print('#######################   localisation de la source GW170814  ###############')
print('#############################################################################')
print('')
print('ascension droite alpha+ = ',alpha_plus, ' soit ',alpha_plusHMS[0],'H',alpha_plusHMS[1],'min')
print('déclinaison delta+ = ',delta_plus)
print('ou')
print('ascension droite alpha- = ',alpha_moins, ' soit ',alpha_moinsHMS[0],'H',alpha_moinsHMS[1],'min')
print('déclinaison delta- = ',delta_moins)